In [ ]:
# Clone YOLOv5 and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
!pip install onnx onnxruntime

Cloning into 'yolov5'...
remote: Enumerating objects: 17771, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 17771 (delta 94), reused 53 (delta 53), pack-reused 17618 (from 2)
Receiving objects: 100% (17771/17771), 17.09 MiB | 16.58 MiB/s, done.
Resolving deltas: 100% (12070/12070), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00


In [ ]:
!unzip /content/Rice-Quality.zip

Archive:  /content/Rice-Quality.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/1700059259412_jpg.rf.86e160c4ccbaac8e6d4093ffb4cfecab.jpg  
 extracting: test/images/1700325971094_jpg.rf.ce5034e23b939ed2f4fed3ad16e88e61.jpg  
 extracting: test/images/20-35-56_c691e2c0_jpg.rf.62a711704f766042fd77fb7c9d6f3a85.jpg  
 extracting: test/images/NewItem107_png.rf.a134e834203c7059db0fc5fa5840fb81.jpg  
 extracting: test/images/c7_jpg.rf.64d0a1c5e2ad94d59f9c81887d78b98c.jpg  
 extracting: test/images/d5_jpg.rf.3a55bc88791fc66b2134fc78297b67ec.jpg  
 extracting: test/images/ds4_jpg.rf.7a93b248aa6def1b2dc17d451f5bbe0d.jpg  
 extracting: test/images/r1_jpg.rf.1835ce71991e71820cf6fc708d2fcfbd.jpg  
 extracting: test/images/r2_jpg.rf.d51c8b4a97cb3bc55fc25b839d15679c.jpg  
 extracting: test/images/rp1_jpg.rf.9f3a556e664f06f71966ddc68337dcb2.jpg  
 extracting: test/

In [6]:
import yaml
with open('data.yaml', 'r') as f:
    data = yaml.safe_load(f)

In [7]:
import os
image_list = os.listdir('test/images/')
print(image_list[:5]) # Shows the first 5 image filenames

['20-35-56_c691e2c0_jpg.rf.62a711704f766042fd77fb7c9d6f3a85.jpg', 'rpk3_jpg.rf.f2c09f5b6d232546e8949bb067eee531.jpg', 'rr3_jpg.rf.7458508f3a6cca422d7ee43b89fa6a7c.jpg', 'vidd_mp4-0000_jpg.rf.59a3e017ffc775e1d789bdbdba167261.jpg', '1700325971094_jpg.rf.ce5034e23b939ed2f4fed3ad16e88e61.jpg']


In [ ]:
import yaml

hyp_path = 'data/hyps/hyp.scratch-low.yaml'

# Load the existing hyperparameter file
with open(hyp_path, 'r') as f:
    hyp = yaml.safe_load(f)

# Update with SOTA Rice-specific values (Minimal Augmentation)
hyp['mosaic'] = 0.5    # Reduce complexity for small grains
hyp['degrees'] = 0.0   # Keep orientation standard
hyp['mixup'] = 0.0     # Avoid "ghost" grains
hyp['fliplr'] = 0.5    # Horizontal flips are safe
hyp['flipud'] = 0.5    # Vertical flips are safe
hyp['hsv_h'] = 0.015   # Minimal color jitter
hyp['hsv_s'] = 0.4     # Minimal saturation jitter
hyp['hsv_v'] = 0.2     # Minimal brightness jitter

# Save the updated file
with open(hyp_path, 'w') as f:
    yaml.dump(hyp, f, default_flow_style=False)

print(f"Successfully updated {hyp_path} for Rice Quality detection.")

Successfully updated data/hyps/hyp.scratch-low.yaml for Rice Quality detection.


In [ ]:
# 1. Increase image size to 1024 to capture rice texture
# 2. Use a much longer decay (patience) so the YOLOv5 Nano can train well
# 3. Use 'hyp.scratch-high.yaml' for best augmentations

!python train.py --img 1024 \
                 --batch 16 \
                 --epochs 300 \
                 --data /content/yolov5/data.yaml \
                 --weights yolov5n.pt \
                 --name rice_nano_distilled_SOTA \
                 --hyp data/hyps/hyp.scratch-low.yaml \
                 --label-smoothing 0.1 \
                 --patience 50

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
    198/299      4.58G     0.0223      0.103    0.04176        555       1024:  36% 8/22 [00:06<00:10,  1.33it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    198/299      4.58G    0.02219     0.1006    0.04188        597       1024:  41% 9/22 [00:07<00:13,  1.04s/it]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    198/299      4.58G     0.0224     0.1013    0.04177        722       1024:  45% 10/22 [00:08<00:10,  1.19it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
  

In [ ]:
'''Model summary: 157 layers, 1769989 parameters, 0 gradients, 4.2 GFLOPs
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 1/1 [00:00<00:00,  1.31it/s]
                   all         14        595      0.819      0.837      0.892      0.759
         broken-chalky         14         16      0.574      0.842       0.79      0.689
          broken-clear         14         83      0.822      0.843      0.853      0.718
        foreign-object         14         22      0.858      0.818      0.923       0.74
               plastic         14         37       0.89      0.877        0.9      0.706
                 sound         14        195      0.814      0.896      0.929      0.811
                 stone         14         40      0.882        0.9      0.934      0.843
               unsound         14        118      0.923      0.809      0.942      0.798
          whole-chalky         14         84      0.789      0.711      0.869      0.764'''